# Using Google's Agent Development Kit (ADK) with Neo4j

This notebook demonstrates how to use Google's Agent Development Kit (ADK) to create a conversational agent that can query a Neo4j database. We will configure an LLM-based agent, connect it to a Neo4j database using the Model Context Protocol (MCP), and then ask it a question about the data.

## 1. Installation

This cell installs the necessary Python libraries:
- `google-adk`: The Google Agent Development Kit.
- `mcp-neo4j-cypher`: The Neo4j MCP server implementation that allows the agent to communicate with the database.

The `-q` flag ensures a quiet installation, minimizing the output.

In [ ]:
%pip install -q google-adk mcp-neo4j-cypher

## 2. Imports

This cell imports all the required modules for the notebook. Grouping imports at the beginning of a script or notebook is a standard practice that improves code readability and helps manage dependencies.

In [2]:
import os
import subprocess
import time
from getpass import getpass

from google.adk.agents.llm_agent import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_session_manager import SseServerParams
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.genai.types import Content, Part

## 3. Configuration

This cell configures the environment variables needed to connect to the Google API and the Neo4j database. You will be prompted to enter your Google API key securely. For the Neo4j database, we are using a publicly accessible demonstration instance.

In [3]:
# Google API
os.environ["GOOGLE_API_KEY"] = getpass("Enter Google API Key: ")
os.environ["GEMINI_MODEL"] = "gemini-flash-latest"

# Neo4j (Aura demo DB)
os.environ["NEO4J_URI"] = "neo4j+s://demo.neo4jlabs.com"
os.environ["NEO4J_USERNAME"] = "companies"
os.environ["NEO4J_PASSWORD"] = "companies"
os.environ["NEO4J_DATABASE"] = "companies"

## 4. Start the MCP Server

This cell starts the Neo4j MCP server as a background process. This server acts as a bridge, translating the agent's requests into Cypher queries that the Neo4j database can understand. We add a short delay to give the server time to initialize before we proceed.

In [4]:
process = subprocess.Popen(
    [
        "mcp-neo4j-cypher",
        "--server-port", "8000",
        "--db-url", os.environ["NEO4J_URI"],
        "--username", os.environ["NEO4J_USERNAME"],
        "--password", os.environ["NEO4J_PASSWORD"],
        "--database", os.environ["NEO4J_DATABASE"],
        "--transport", "sse",  
    ]
)

# Allow time for the server to start
time.sleep(5)

/workspaces/neo4j-agent-integrations/.venv/lib/python3.12/site-packages/fastmcp/server/server.py:286: DeprecationWarning: Providing `stateless_http` when creating a server is deprecated. Provide it when calling `run` or as a global setting instead.
  self._handle_deprecated_settings(


╭──────────────────────────────────────────────────────────────────────────────╮
│                                                                              │
│                                                                              │
│                         ▄▀▀ ▄▀█ █▀▀ ▀█▀ █▀▄▀█ █▀▀ █▀█                        │
│                         █▀  █▀█ ▄▄█  █  █ ▀ █ █▄▄ █▀▀                        │
│                                                                              │
│                                                                              │
│                                FastMCP 2.14.3                                │
│                            https://gofastmcp.com               

## 5. Define the MCP Toolset

This cell creates an `McpToolset`, which is the tool the agent will use to interact with the MCP server. It's configured with the URL of the local server we just started.

In [5]:
neo4j_mcp_toolset = McpToolset(
    connection_params=SseServerParams(
        url="http://localhost:8000/mcp"
    )
)

## 6. Create the Agent

Here, we define our `LlmAgent`. We specify the language model to use (`gemini-flash-latest`), give it a name, and provide a system instruction. Most importantly, we equip it with the `neo4j_mcp_toolset`, enabling it to query the database.

In [6]:
agent = LlmAgent(
    model="gemini-flash-latest",
    name="neo4j_agent",
    instruction=(
        "You can inspect the Neo4j schema and run Cypher queries. "
        "Always use the Neo4j MCP tool when answering database questions."
    ),
    tools=[neo4j_mcp_toolset],
)

## 7. Set Up the Runner and Session

This cell sets up the `Runner`, which manages the conversation flow. It also creates an in-memory session to maintain the context of the interaction between the user and the agent.

In [7]:
# Create session service
session_service = InMemorySessionService()

# Create the Runner
runner = Runner(
    agent=agent,
    app_name="neo4j_app",
    session_service=session_service
)

# Create a session
session_id = "session1"
user_id = "user1"
await session_service.create_session(
    app_name="neo4j_app",
    user_id=user_id,
    session_id=session_id
)

Session(id='session1', app_name='neo4j_app', user_id='user1', state={}, events=[], last_update_time=1769525138.3711772)

## 8. Run the Agent and Ask a Question

This final cell sends a question to the agent. The `runner.run()` method streams the agent's response. We process the events to capture the final text answer and print it.

### Note

Executing the cell below will produce the correct answer, but it will also be followed by a stack trace containing `RuntimeError` and `ExceptionGroup` errors.

They stem from an issue within the underlying `google-adk` and `mcp-client` libraries related to the cleanup of asynchronous connections (SSE). The agent successfully retrieves the data, but the library fails to shut down the connection cleanly, leading to the error messages during the final cleanup phase. 

In [ ]:
user_message = Content(role="user", parts=[Part(text="How many people are in the database?")])

final_text = None
try:
    for event in runner.run(
        user_id=user_id,
        session_id=session_id,
        new_message=user_message
    ):
        if event.is_final_response() and event.content and event.content.parts:
            # ADK returns text in the Content parts
            final_text = "".join(
                [p.text or "" for p in event.content.parts if p.text is not None]
            )
    print(final_text)
except (RuntimeError, ExceptionGroup):
    pass
finally:
    # Allow time for graceful shutdown before terminating the process
    time.sleep(1)
    process.terminate()
    print("MCP server terminated.")

INFO:     127.0.0.1:50620 - "GET /mcp HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:50620 - "GET /mcp/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:50632 - "POST /messages/?session_id=2e569871c1ac472dacd520814866f550 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:50632 - "POST /messages/?session_id=2e569871c1ac472dacd520814866f550 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:50632 - "POST /messages/?session_id=2e569871c1ac472dacd520814866f550 HTTP/1.1" 202 Accepted


/workspaces/neo4j-agent-integrations/.venv/lib/python3.12/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:88: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
/workspaces/neo4j-agent-integrations/.venv/lib/python3.12/site-packages/google/adk/flows/llm_flows/base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:


INFO:     127.0.0.1:50632 - "POST /messages/?session_id=2e569871c1ac472dacd520814866f550 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:50632 - "POST /messages/?session_id=2e569871c1ac472dacd520814866f550 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:50632 - "POST /messages/?session_id=2e569871c1ac472dacd520814866f550 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:50632 - "POST /messages/?session_id=2e569871c1ac472dacd520814866f550 HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:50632 - "POST /messages/?session_id=2e569871c1ac472dacd520814866f550 HTTP/1.1" 202 Accepted


an error occurred during closing of asynchronous generator <async_generator object aconnect_sse at 0x77b33326f230>
asyncgen: <async_generator object aconnect_sse at 0x77b33326f230>
Traceback (most recent call last):
  File "/workspaces/neo4j-agent-integrations/.venv/lib/python3.12/site-packages/httpx_sse/_api.py", line 75, in aconnect_sse
    yield EventSource(response)
GeneratorExit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/workspaces/neo4j-agent-integrations/.venv/lib/python3.12/site-packages/httpx_sse/_api.py", line 74, in aconnect_sse
    async with client.stream(method, url, headers=headers, **kwargs) as response:
  File "/usr/lib/python3.12/contextlib.py", line 267, in __aexit__
    raise RuntimeError("generator didn't stop after athrow()")
RuntimeError: generator didn't stop after athrow()
an error occurred during closing of asynchronous generator <async_generator object sse_client at 0x77b333355b90>
asyncge

There are 8,064 people in the database.
MCP server terminated.


INFO:     Shutting down
ERROR:    Cancel 0 running task(s), timeout graceful shutdown exceeded
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [44170]
